In [79]:
import pandas as pd
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [80]:
test = pd.read_csv("./data/final.tsv", sep = "\t", header = None, quoting=csv.QUOTE_NONE)

In [81]:
test.columns = "context_id,context_2,context_1,context_0,reply_id,reply".split(",")

In [82]:
test = test.replace(np.nan, '', regex=True)

In [83]:
test['text'] = test['context_2'] + " " + test['context_1'] + ' ' + test['context_0'] + ' ' + test['reply']

In [84]:
df = pd.read_csv("./data/train.tsv", sep = "\t", header = None, quoting=csv.QUOTE_NONE)

In [85]:
df.columns = "context_id,context_2,context_1,context_0,reply_id,reply,label,confidence".split(",")

In [86]:
df = df.replace(np.nan, '', regex=True)

In [87]:
df['text'] = df['context_2'] + " " + df['context_1'] + ' ' + df['context_0'] + ' ' + df['reply']

In [88]:
def get_target(x):
    if x == 'good':
        return 3
    if x == 'bad':
        return 1
    if x == 'neutral':
        return 2

In [89]:
df['target'] = df['label'].apply(lambda x: get_target(x))

In [133]:
tf = TfidfVectorizer(ngram_range=(1, 1), max_features = 50000)
df_train = tf.fit_transform(df['text'].values)
tf_idf = tf.transform(test['text'])

In [134]:
train_df, y = df_train, df['target']

In [135]:
skf = StratifiedKFold(n_splits=5, shuffle=True)

In [136]:
svc = LinearSVC(loss='hinge', random_state=17,)

In [137]:
svc.fit(train_df, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=17, tol=0.0001, verbose=0)

In [138]:
predicted = svc.predict(tf_idf)

In [139]:
test['target'] = predicted
test['target'] = test['target'].astype('int')
test = test.sort_values(['context_id', 'target'], ascending=[True, False])
test[['context_id','reply_id']].to_csv('submission.tsv', index=False, header=None, sep='\t')